# O-modes in a periodic PIC simulation

We need to find the perturbed quantities associated with o-mode waves, 

The procedure is:
 - Fix $\omega$, $\Omega_s$, $\Pi_s$ and $\theta_k$ to find $k_\parallel$ and $k_\bot$.
 - Use the nullspace, or vector associated with the singular value, of the dispersion relation matrix $\Lambda$ to obtain the electric field polarisation for each of the waves.
 - Insert the electric field into Faraday's law for the magnetic field polarisation
 - Obtain the current polarisation from the conductivity tensor, $\vec J_s = \sigma_s \cdot \vec E$
 - Obtain the velocity perturbation for each species from their currents.
 - Obtain the density perturbation for each species from the continuity equation.

The conductivity tensor $\sigma$ is defined for species $s$
$$
\sigma_s = \begin{bmatrix}
  \frac{\omega}{\omega^2 - \Omega_s^2} & \frac{i \Omega_s}{\omega^2 - \Omega_s^2} & 0\\
  -\frac{i \Omega_s}{\omega^2 - \Omega_s^2} & \frac{\omega}{\omega^2 - \Omega_s^2} & 0\\
   0 & 0 & \frac{1}{\omega} 
   \end{bmatrix} i \epsilon_0  \Pi_s^2
$$

The dielectric tensor is then $\varepsilon = I + \frac{i}{\epsilon_0 \omega} \sum_s \sigma_s$, which makes the dispersion relation tensor $\Lambda = (\vec{k} \vec{k}' - I(\vec k \cdot\vec k))\frac{c^2}{\omega^2} + \varepsilon$, for which $\vec k = |k| [\sin\theta, 0, \cos\theta]^{T}$ ($\theta=\pi/2$ corresponds to perpendicular progation studied here, meaning that $\vec k$ is aligned along x and $\vec B$ is aligned along z, even in the EPOCH simulation; $\theta$ changes from zero then care must be taken to project the polarisation into the coordinate system in which $\vec k$ remains parallel to $x$ but $\vec B$ has a nonzero $x$ component).

Now the other parameters:

Faraday's
$$
-i\omega\vec B_1 = - \vec k \times \vec E_1
$$
Gauss's law
$$
i\vec k \cdot \vec E_1 =  \sum_s\frac{q_s n_{s,1}}{\epsilon_0}\\
$$
Ampere's law:
$$
\mu_0 \vec J = i \vec k \times \vec {B}_1 + i \omega \mu_0 \epsilon_0 \vec E_1\\
\sum_s q_s n_{s,0} \vec {v}_{s,1} = \frac{i \vec k \times \vec {B}_1}{\mu_0} + i \omega \epsilon_0 \vec E_1\\
$$
Assume the ions are unperturbed in the initial conditions
$$
q_e n_{e,0} {v}_{e,1,y} = i \omega \epsilon_0 E_{1,x}\\
q_e n_{e,0} {v}_{e,1,y} = -\frac{i k_x {B}_{1,z}}{\mu_0} + i \omega \epsilon_0 E_{1,y}\\
q_e n_{e,0} {v}_{e,1,z} = \frac{i k_x {B}_{1,y}}{\mu_0} + i \omega \epsilon_0 E_{1,z}\\
$$

$$
n_{e,1} = i k_x E_{x,1}\frac{\epsilon_0}{q_s}
$$

Continuity

$$
\omega n_{1,s} - n_{0,s} \vec k_x \cdot \vec v_{1,s} = 0
$$


In [1]:
using Roots, Plots, LinearAlgebra, Dates

const c₀ = 2.99792458e8
const ϵ₀ = 8.854187817e-12
const μ₀ = 1.25663706144e-6
const q₀ = 1.602176487e-19
const mₑ₀ = 9.10938188e-31
const mi = 1836 * mₑ₀ * 2
const memultiplier = 1
const mₑ = mₑ₀ * memultiplier
const n0 = 1e18
const B0 = 2.0
fΠ(n, m::Float64=mₑ) = √(q₀^2 * n / m / ϵ₀)
const Π0 = fΠ(n0)
fΩ(m=mₑ, Z::Int=-1) = Z * q₀ * B0 / m
const Ωe = fΩ(mₑ, -1)
const Ωi = fΩ(mi, 1)
# fωL(n) = sqrt(Ωe^2 / 4 + fΠ(n)^2) + Ωe / 2
# const ωL0 = fωL(n0)
# fωₕ(n) = √(fΠ(n)^2 + Ωe^2)
# const ωₕ0 = √(fΠ(n0)^2 + Ωe^2)
const Te_eV = 10e3
const Ti_eV = 10e3
const vthe = sqrt(2 * Te_eV * q₀ / mₑ)
fλD(n) = vthe / fΠ(n)
const λD0 = fλD(n0)
const fbulkmagneticfieldenergydensity(n) = B0^2 / 2μ₀
const θ₀ = 40 * pi / 180
# const fbulkenergydensity(n) = fbulkmagneticfieldenergydensity(n) + n * Te_eV * q₀ + n * Ti_eV * q₀

function cleaner(z)
  nz = norm(z)
  normz = z ./ nz
  normz = trunc.(real.(normz), digits=16) .+ im * trunc.(imag.(normz), digits=16)
  output = normz .* nz
  @assert output ≈ z
  return output
end

function fσ(ω, n, m=mₑ, Z=-1)
  Ωs = fΩ(m, Z) 
  Πs = fΠ(n, m)
  ω²_Ωs² = (ω^2 - Ωs^2)
  return [ω / ω²_Ωs²         im * Ωs / ω²_Ωs²  0.0  ; 
          -im * Ωs / ω²_Ωs²  ω / ω²_Ωs²        0.0  ;
          0.0                0.0               1 / ω] .* im * ϵ₀ * Πs^2
end
fϵ(ω, n) = I(3) + im * (fσ(ω, n, mₑ, -1) + fσ(ω, n, mi, 1)) / (ω * ϵ₀)
fkvector(k) = k * [sin(θ₀), 0, cos(θ₀)]
fΛ(ω, k::Number, n) = fΛ(ω, fkvector(k), n)
function fΛ(ω, k, n)
    N = k * c₀ / ω
    output = N * N' .- dot(N, N) * I(3) .+ fϵ(ω, n)
    # @show abs(det(output))
    return output
end
function fE(ω, k, n)
  Λ = fΛ(ω, k, n)
  _, sigma, V = svd(Λ)
  @assert sigma[3] <= sqrt(eps()) "sigma[3] = $(sigma[3]), det(Λ) = $(det(Λ))"
  E = V[:, 3]
  @assert all(E .≈ nullspace(Λ))
  @assert all(abs.(Λ * E) .<= sqrt(eps()))
  return E
end
absdetΛ(ω, k, n=n0) = abs(det(fΛ(ω, k, n)))

fB(ω, k, n) = cross(fkvector(k), fE(ω, k, n)) / ω
fJ(ω, k, n, m::Float64, Z::Int) = fσ(ω, n, m, Z) * fE(ω, k, n)
fJ(ω, k, n) = fJ(ω, k, n, mₑ, -1) + fJ(ω, k, n, mi, 1)
fV(ω, k, n, m::Float64=mₑ, Z::Int=-1) = fJ(ω, k, n, m, Z) / q₀ / n / Z
fn(ω, k, n, m::Float64=mₑ, Z::Int=-1) = n * dot(fkvector(k), fV(ω, k, n, m, Z)) / ω


fn (generic function with 3 methods)

In [2]:
# vthi = sqrt(Te_eV * q₀ * 2 / mi)
# rli = vthi / Ωi
# vthe = sqrt(Te_eV * q₀ * 2 / mₑ)
# rle = vthe / abs(Ωe)
function solvefork(ω = 140e9 * 2π)
  k = Roots.fzero(k->absdetΛ(ω, k, n0), ω / c₀, fatol=eps())
  @show k
  @show ω / k / c₀
  @show ω / fΠ(n0, mₑ)
  @show ω / k / vthe
  (k⊥, _, kz) = fkvector(k)
  return k
end

# Lx = 2π / k * 10

solvefork (generic function with 2 methods)

In [3]:
permuterkzk⊥_kxkykz(x) = [x[3], x[1], x[2]]

function checkpolarisation(ω, k, polarisation)
  E = polarisation[:E]
  B = polarisation[:B]
  J = polarisation[:J]
  ve = polarisation[:ve]
  vi = polarisation[:vi]
  ne = polarisation[:ne]
  ni = polarisation[:ni]
  kvec = polarisation[:kvec]
  @assert abs(dot(kvec, B)) < norm(k) * norm(B) * 1e-8 # Monopoles
  @assert all(ω * B ≈ cross(kvec, E)) # Faraday
  @assert isapprox(-im * J, cross(kvec, B) / μ₀ + ω * ϵ₀ * E, rtol=1e-6) # Ampere
  @assert isapprox(im * dot(kvec, E) * ϵ₀, q₀ * (ni - ne), rtol=1e-6) # Gauss
  @assert isapprox(im * dot(kvec, J), im * q₀ * (ni - ne) * ω, rtol=1e-6) # divJ + drho/dt = 0 
  @assert all(J ≈ q₀ * n0 * (vi - ve)) # current
end

function findpolarisation(ω, k, n)
  Ep = permuterkzk⊥_kxkykz(cleaner(fE(ω, k, n)))
  Bp = permuterkzk⊥_kxkykz(cleaner(fB(ω, k, n)))
  Jp = permuterkzk⊥_kxkykz(cleaner(fJ(ω, k, n)))
  ve = permuterkzk⊥_kxkykz(cleaner(fV(ω, k, n, mₑ, -1)))
  vi = permuterkzk⊥_kxkykz(cleaner(fV(ω, k, n, mi, 1)))
  ne = cleaner(fn(ω, k, n, mₑ, -1))
  ni = cleaner(fn(ω, k, n, mi, 1))
  kvec = permuterkzk⊥_kxkykz(fkvector(k))
  pol = (E=Ep, B=Bp, J=Jp, ve=ve, vi=vi, ne=ne, ni=ni, kvec=kvec)
  checkpolarisation(ω, k, pol)
  return pol
end


findpolarisation (generic function with 1 method)

In [4]:
const ignorekeys = [:kvec, :ω]
function scale(polarisation, factor::Real)
  return NamedTuple(k=> k in ignorekeys ? v : factor * v for (k, v) in pairs(polarisation))
end
function zeroions(polarisation)
  return NamedTuple(k=>!(k in (:vi, :ni)) * v for (k, v) in pairs(polarisation))
end
function energy(polarisation)
  E = polarisation[:E]
  B = polarisation[:B]
  ve = polarisation[:ve]
  vi = polarisation[:vi]
  return real(ϵ₀ * dot(E,E) / 2 + dot(B,B) / 2μ₀ +
            mₑ * n0 * dot(ve,ve) / 2 + mi * n0 * dot(vi,vi) / 2)
end
printer(polarisation, i) = printer(stdout, polarisation, i)

function printcommon(io, pol)
  kx, ky, kz = pol[:kvec]
  λx = 2π / kx
  λy = 2π / ky
  println(io, "n0 = $n0")
  println(io, "B0 = $B0")
  println(io, "lambdadebye = $λD0")
  cellx = min(λD0, λx / 2)
  celly = min(λD0, λy / 2)
  Lx = ceil(0.2 / λx) * λx
  Ly = ceil(0.2 / λy) * λy
  NGX = ceil(Int, Lx / cellx)
  NGY = ceil(Int, Ly / celly)
  # println(io, "Lx = $Lx")
  println(io, "NGX = $NGX")
  println(io, "NGY = $NGY")
  println(io, "Te_eV = $Te_eV")
  println(io, "Ti_eV = $Ti_eV")
  println(io, "mioverme = $(round(Int, mi / mₑ))")
  # println(io, "waveenergytobulkmagneticfieldenergyratio = $energyratio")
end
function printscalar(io, scalar, str)
  println(io, str * "_cos = $(real(scalar))")
  # Minus sign because real((a + im*b) * exp(im *k*x)) = a*cos(k*1x) - b*sin(k*x).
  println(io, str * "_sin = $(-imag(scalar))")
end
function printvector(io, vec, str)
  components = ["x", "y", "z"]
  for i in 1:3
    printscalar(io, vec[i], str * components[i])
  end
end

function printer(io, polarisation)
  printvector(io, polarisation[:E], "E")
  printvector(io, polarisation[:B], "B")
  printvector(io, polarisation[:J], "J")
  printvector(io, polarisation[:ve] * mₑ, "Pe")
  printvector(io, polarisation[:vi] * mi, "Pi")
  printscalar(io, polarisation[:ne], "ne")
  printscalar(io, polarisation[:ni], "ni")
end


printer (generic function with 1 method)

In [5]:
ω = 10Ωi #140e9 * 2π
k = solvefork(ω)
pol = findpolarisation(ω, k, n0)
@assert norm(pol[:E]) ≈ 1 norm(pol[:E])# by convention

bfieldenergydensity = fbulkmagneticfieldenergydensity(n0)
# energyratio = 1e-4
# checkpolarisation(ω, k, pol) # make sure they're still consitent
# @show sqrt(bfieldenergydensity * energyratio / energy(pol))
# energyscale = sqrt(bfieldenergydensity * energyratio / energy(pol)) # how to scale energy
pol = scale(pol, 1e4)
@show norm(pol[:E])
# @assert energy(pol) ≈ energyratio * bfieldenergydensity
checkpolarisation(ω, k, pol) # make sure they're still consitent
kx, ky, _ = pol[:kvec]
@show pol

open("null.prefix.deck","w") do io
  println(io, "begin:constant")
  printcommon(io, pol)
  println(io, "# Wave")
  println(io, "w = $ω")
  println(io, "k = $k")
  println(io, "kx = $kx")
  println(io, "ky = $ky")
  printer(io, scale(pol, 0))
end

open("wave.prefix.deck","w") do io
  println(io, "begin:constant")
  printcommon(io, pol)
  println(io, "# Wave")
  println(io, "w = $ω")
  println(io, "kx = $kx")
  println(io, "ky = $ky")
  printer(io, pol)
end

@info "decks written at $(now())"

k = 11.324627367918831
(ω / k) / c₀ = 0.28216565113086745
ω / fΠ(n0, mₑ) = 0.016980764298206927
(ω / k) / vthe = 1.4262613950373726
norm(pol[:E]) = 9999.999999999998
pol = (E = ComplexF64[14.150318993634999 - 0.0im, 7946.594028932778 - 0.0im, -1.1999999999999997e-10 + 6070.538947062465im], B = ComplexF64[-9.038300184365229e-19 + 4.6128560162753696e-5im, 1.086627100816943e-18 - 5.497387729515988e-5im, 7.18556218852791e-5 + 0.0im], J = ComplexF64[0.0 + 416.3585783096637im, 9.69958601127517e-12 - 428.6509028323116im, 595.2300285130685 - 9.27786488035016e-13im], ve = ComplexF64[-0.0 - 2597.998567231426im, -5.985150637281672e-11 + 3046.112590437706im, -3981.5925280569063 - 0.0im], vi = ComplexF64[0.0 + 0.7075159496817557im, 5.93471257186084e-13 + 370.6838451629561im, -266.4585628368276 - 6.026015842197161e-12im], ne = -0.4566387926577154 - 3.804055253729718e11im, ni = 0.0045170307539182985 + 2.8231442211989365e12im, kvec = [8.67516786558732, 7.27933015641531, 0.0])


[ Info: decks written at 2024-10-04T12:01:13.614
